# **Práctica 9: Árboles. Parte 2. (Descripción General de Funcionamiento)**

In [ ]:
import matplotlib.pyplot as plt # para graficar las complejidades
import random # para las pruebas con grandes volúmenes de datos
import string # para generar cadenas de caracteres en cada "nodo" del arbol

class Data: # contenedor para almacenar los datos que irán dentro del Árbol

    def __init__(self, key, value): # constructor
        self.key = key # para ordenar y buscar en el arbol
        self.value = value # atributo asociado a la llave

    def __str__(self): # representación del objeto
        return str(self.key) + "->" + self.value # formato

class BTreeNode: # representación de un nodo individual dentro del Árbol
    
    def __init__(self, t, leaf): # constructor
        self.t = t # grado mínimo del Árbol, este parámetro define cuantas llaves (entre t-1 y 2t-1) y cuantos hijos (entre t y 2t) puede tener un nodo
        self.leaf = leaf # booleano que indica si el nodo es una hoja o un nodo interno
        self.keys = [None] * (2 * self.t - 1) # inicializa una lista para almacenar las llaves (objetos Data) del nodo, su tamaño máximo es 2t-1
        self.C = [None] * (2 * self.t) # inicializa una lista para almacenar las referencias a los nodos hijos, su tamaño  máximo es 2t
        self.n = 0 # contador que lleva la cuenta de cuántas llaves están actualmente almacenadas en el nodo

    def insertNonFull(self, data, time, space): # para insertar un nuevo objeto Data en un nodo que se sabe que no esta lleno (self.n < 2t-1)
        time += 1 
        space += 1
        i = self.n - 1 # si no está lleno
        if self.leaf: # si es un nodo hoja
            while i >= 0 and self.keys[i].key > data.key: # encuentra la posición correcta para data.key moviendo todas las llaves mayores que ella una posición derecha.
                time += 1
                self.keys[i + 1] = self.keys[i]
                i -= 1

            self.keys[i + 1] = data # inserción de data en la posición correcta
            self.n += 1 # cantidad de llaves que están actualmente en el nodo

        else: # si es un nodo interno
            while i >= 0 and self.keys[i].key > data.key: # encuentra el hijo (self.C[i + 1]) por el que debe desender para insertar Data
                time += 1
                i -= 1

            if self.C[i + 1].n == 2 * self.t - 1: # si ese hijo está lleno
                ts, ss = self.splitChild(i + 1, self.C[i + 1]) # llama a splitChild para dividirlo, esto promueve una llave al nodo actual y crea dos hijos a partir del hijo lleno
                time += ts
                space += ss

                if self.keys[i + 1].key < data.key: # cual de los dos dos hijos nuevo (o el hijo original si no hubo división) es el correcto para insertar Data
                    i += 1
            time, space = self.C[i + 1].insertNonFull(data, time, space) # llama recursivamente a insertNonFull en ese hijo apropiado
        return time, space   

    def splitChild(self, i, y): # divide un nodo HIJO (que está en self.C[i]) que está lleno
        time, space = 0, 
        z = BTreeNode(y.t, y.leaf) # crea un nodo que será HERMANO del nodo HIJO 
        time += 1
        space += 1
        z.n = self.t - 1
        for j in range(self.t - 1): # mueve la mitad superior de las llaves del HIJO al HERMANO
            z.keys[j] = y.keys[j + self.t]

        if not y.leaf: # si HIJO no era hoja, mueve la mitad superior de los hijos de HIJO a HERMANO
            for j in range(self.t):
                z.C[j] = y.C[j + self.t]
                
        y.n = self.t - 1 # se toma la llave median de HIJO (la que esta en y.keys[t-1])
        for j in range(self.n, i, -1): # y la promueve al nodo actual (self)
            self.C[j + 1] = self.C[j] # insertándola en self.keys[i+1]

        self.C[i + 1] = z # inserta el nodo HERMANO como hijo del nodo actual (self) en la posición self.C[i+1]
        
        for j in range(self.n - 1, i - 1, -1): # actualiza los contadores n(cantidad de llaves que están actualmente en el nodo) de los tres nodos involucrados (self, HIJO y HERMANO)
            time += 1
            self.keys[j + 1] = self.keys[j]
        self.keys[i] = y.keys[self.t - 1]
        self.n += 1
        return time, space
        
    def traverse(self, l, time, space): # recorre el subárbol que comienza en este nodo
        for i in range(self.n): # itera de 0 a n - 1
            time += 1
            space += 1

            if not self.leaf: # si no es hoja
                time, space = self.C[i].traverse(l + 1, time, space) # se llama recursivamente en el hijo (self.C[i])
            # print("\t" * l, l, self.keys[i], end=' ') # visita a la llave y la imprime
        # print()

        if not self.leaf: # si no es hoja
            time, space = self.C[self.n].traverse(l + 1, time, space) # se llama recursivamente en el último hijo
        return time, space
        
    def search(self, k, time, space): # busca una llave k en el subárbol que comienza en este nodo
        time += 1
        space += 1
        i = 0
        while i < self.n and k > self.keys[i].key: # encuentra el primer índice i tal que k <= self.keys[i].key
            time += 1
            i += 1
        if i < self.n and k == self.keys[i].key: # si encuentra la llave exacta (k == self.keys[i].key), devuelve el nodo actual (self)
            return self, time, space
        if self.leaf: # si no la encuentra y el nodo es una hoja, la llave no está en el árbol
            return None, time, space # devuelve None.
        return self.C[i].search(k, time, space) # si no la encuentra y el nodo no es una hoja, llama recursivamente a search en el hijo apropiado (self.C[i])

class BTree: # representación del Árbol completo y actúa como interaz principal
    
    def __init__(self, t): # constructor
        self.root = None # inicializa la raiz del árbol
        self.t = t # grado mínimo del árbol
    
    def traverse(self): # inicia el recorrido completo del árbol desde la raíz
        if self.root != None: # si la raíz no es None, llama al método traverse del nodo raíz
            time, space = self.root.traverse(0, 0, 0)
            return time, space
        return 0, 0
    
    def search(self, k): # inicia la búsqueda de la llave k en todo el árbol
        if self.root is None: # si la raíz es None
            return None, 0, 0 # devuelve None´

        else: # si la raíz no es None
            result, time, space = self.root.search(k, 0, 0) # llama al método search del nodo raíz
        return result, time, space
    
    def insert(self, data): # método principal para insertar un nuevo Data en el árbol
        time, space, t_split, s_split = 1, 1, 1, 1
        if self.root == None: # si el árbol está vació 
            self.root = BTreeNode(self.t, True) # crea un nuevo nodo
            self.root.keys[0] = data # lo asigna como la nueva raíz
            self.root.n = 1 # se actualiza la cantidad de llaves en el nodo

        else: # tiene raíz
            if self.root.n == 2 * self.t - 1: # si la raíz actual esta llena
                s = BTreeNode(self.t, False) # crea un nuevo nodo que será la NUEVA raíz (y no será una hoja)
                s.C[0] = self.root # hace que la ANTIGUA raíz sea el primer hijo de esta NUEVA raíz
                t_split, s_split = s.splitChild(0, self.root) # divide la ANTIGUA raíz, esto promueve la llave mediana de la ANTIGUA raíz a la NUEVA raíz, ahora la NUEVA tiene 1 llave y 2 hijos
                i = 0

                if s.keys[0].key < data.key: # determina cual de los 2 hijos es el correcto para insertar Data
                    i += 1

                time, space = s.C[i].insertNonFull(data, 0, 0) # llama a insertar en ese hijo
                self.root = s # ahora es la nueva raíz del árbol

            else: # si la raíz o está llena
                time, space = self.root.insertNonFull(data, 0, 0) # llama a insertar el Data
        return time, space, t_split, s_split